In [47]:
import numpy as np
from skimage import io, util, measure
from scipy.ndimage import gaussian_filter
from scipy import interpolate
import pandas as pd

from os import path
import stackview
import matplotlib.pyplot as plt


# 0. Load datasets

In [48]:
# Load segmentations
dirname = dirname = '/Users/xies/Library/CloudStorage/OneDrive-Stanford/Skin/Two photon/Shared/K10 paw/K10-R2/Uncropped'

masks = io.imread(path.join(dirname,'R_cp_masks_basal.tif'))
heightmap = io.imread(path.join(dirname,'heightmap.tif'))
height_image = io.imread(path.join(dirname,'height_image.tif'))

In [51]:
df = pd.DataFrame(measure.regionprops_table(masks,properties={'label','area','centroid','bbox'}))

# Look at each XY coord and look up heightmap
Z = heightmap[ df['centroid-1'].round().astype(int), df['centroid-2'].round().astype(int) ]
df['Corrected Z'] = df['centroid-0'] - Z


In [52]:
Z

array([79, 78, 83, ..., 62, 82, 70], dtype=int64)

In [56]:
stackview.switch([masks.astype(float),height_image],
                 colormap=['pure_magenta','pure_green'],
                zoom_factor=0.5,
                toggleable=True)

In [59]:
df.to_excel(path.join(dirname,'pre_gated.xlsx'))

cluster_plot = stackview.clusterplot(image=masks.astype(float),
                     labels=masks,
                     df=df,
                     column_x="area",
                     column_y="Corrected Z", 
                     zoom_factor=0.5,
                     alpha=0.5)
cluster_plot

In [66]:
basals = df[df['selection']]
basal_masks = masks.flatten()
I = np.in1d(basal_masks, basals['label'])
basal_masks[~I] = 0
basal_masks = basal_masks.reshape(masks.shape)

/var/folders/zp/5k1kvm9j0t19cvq8h628kr8h0000gn/T/ipykernel_91099/2611368880.py:3: DeprecationWarning: `in1d` is deprecated. Use `np.isin` instead.
  I = np.in1d(basal_masks, basals['label'])


In [67]:
stackview.switch({'all_masks':masks,
                  'basals':basal_masks.astype(float)},
                 colormap=['pure_magenta','pure_green'],
                 toggleable=True,
                 zoom_factor=0.5)

In [ ]:
io.imsave(path.join(dirname,'R_cp_masks_cleaned.tif'),basal_masks)